In [1]:
from datascience import *
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 32: Likelihood Ratio Tests

Last time, we introduced Likelihood Ratio tests. Recall that the point of a likelihood ratio test is to compare the likelihood function under a hypothesized value of the parameter with the liklihood function at its maximum. Instead of looking at the ratio $\Lambda$ itself, we often consider $-2\log \Lambda$ instead, since it has a handy distribution. 

### Example 1: Exponential Distribution

Suppose $X_1,X_2,...,X_n$ is an iid sequence of random variables from the exponential distribution with unknown parameter $\lambda$. Recall that the maximum likelihood estimate of $\lambda$ is $1\over\bar{X}$. We collect a random sample of size 20 and want to test the hypothesis $H_0: \lambda = 3$ vs $H_1: \lambda \neq 3$. Using the data in the python box below, conduct a likelihood ratio test on this hypothesis.  

In [2]:
my_data=np.array([0.18,0.277,0.105,0.126,0.225,0.026,0.123,0.423,0.006,0.281,0.050,0.692,0.105,0.275,0.346,0.079,0.045,0.222,0.063,0.281])

In [3]:
#describe the data
stats.describe(my_data)

DescribeResult(nobs=20, minmax=(0.006, 0.692), mean=0.1965, variance=0.027176368421052633, skewness=1.3751532772375619, kurtosis=2.0594082496235426)

In [4]:
#Start by finding ML of the data
mu=np.mean(my_data) #mean of the data
Lambda_ML=1/(np.mean(my_data))
print(Lambda_ML)

5.089058524173028


In [5]:
Likelihood_ML=np.prod(Lambda_ML*e**(-Lambda_ML*my_data)) #one way to do it
print(Likelihood_ML)
Likeilhood_ML_2=Lambda_ML**20*e**(-20*Lambda_ML*mu) #Another way to do it
print(Likeilhood_ML_2)
Likelihood_3=np.prod(3*e**(-3*my_data))
Capital_Lambda=Likelihood_3/Likelihood_ML
print(Capital_Lambda)

279807.44620464457
279807.4462046449
0.09445694279678163


In [9]:
print('Test statistic of -2log(Lambda):',-2*log(Capital_Lambda))

Test statistic of -2log(Lambda): 4.719222360188457


In [10]:
#Now our next question is: what is the probability we get a test statistic of above?
p_value=stats.chi2.sf(-2*log(Capital_Lambda),1) #survival function (equivalent to 1-cdf)
p_value

0.02982722919477517

#### Power

Suppose that the true value of $\lambda$ is 5. Let's determine the power of this test. Let $n=20$. Then determine the power if $n=50$. Remember, power is the probability of correctly rejecting the null hypothesis. 

First, find what value of $-2 \log \Lambda$ would lead you to reject $H_0$. This is sometimes called the critical value. 

In [11]:
critical_value=stats.chi2.ppf(.95,1)
critical_value

3.841458820694124

Next, obtain the power. Obtain a sample of size 20 from the true population and obtain the value of $-2\log \Lambda$ for this sample. Repeat many times and determine how often you reject the null hypothesis. 

In [27]:
Reject=[]
for i in np.arange(100000):
    minisample=stats.expon.rvs(size=20,scale=1/5) #This is considered the true population
    mu_sample=np.mean(minisample)
    lamb_sample=1/mu_sample
    Likelihood_3=np.prod(3*e**(-3*minisample))
    Likelihood_ML=np.prod(lamb_sample*e**(-lamb_sample*minisample))
    Reject=np.append(Reject,-2*log(Likelihood_3/Likelihood_ML)>=critical_value)
np.mean(Reject)

0.59774

We conclude that for a sample of size 20, we correctly reject the null around 60 percent of the time

Repeat for a sample size of 50. What do you expect to happen to power? 

We expect to reject the null significantly more times than with a sample size of 20. As a result, we expect to correctly reject the null a far greater percent of the time compared to a sample with a size of 20. As a result, the power increases.

In [28]:
Reject=[] 

for i in np.arange(10000):
    minisample=stats.expon.rvs(size=50,scale=1/5) #This is considered the true population (size here is 50)
    mu_sample=np.mean(minisample)
    lamb_sample=1/mu_sample
    Likelihood_3=np.prod(3*e**(-3*minisample))
    Likelihood_ML=np.prod(lamb_sample*e**(-lamb_sample*minisample))
    Reject=np.append(Reject,-2*log(Likelihood_3/Likelihood_ML)>=critical_value)
np.mean(Reject)

0.9523

Since this is a hard topic to concieve, let's do another test and let $\lambda$ be 5, that of the true value of the population and determine how many times we reject this "new" test statistic. That is, for the purposes of argument, let $H_0=5$ and perform the statistic as above. 

In [29]:
Reject=[] 

for i in np.arange(10000):
    minisample=stats.expon.rvs(size=50,scale=1/5) #This is considered the true population (size here is 50)
    mu_sample=np.mean(minisample)
    lamb_sample=1/mu_sample
    Likelihood_5=np.prod(5*e**(-5*minisample)) #Note the change
    Likelihood_ML=np.prod(lamb_sample*e**(-lamb_sample*minisample))
    Reject=np.append(Reject,-2*log(Likelihood_5/Likelihood_ML)>=critical_value)
np.mean(Reject)

0.0492

Interesting, we've only rejected this "new" $H_0$ around 5% of the time. Which, when looking above, given a true value of the system and comparing it, we've found ourselves computing the probability of a Type II error, that is, $\beta$ or how many times we've incorrectly rejected the null when the null was true. This is almost starting to make sense... but not quite yet..

### A Different Test

We've explored hypothesis tests in this class before. Taking advantage of our computing power, we don't have to rely on test statistics with asymptotic distributions. Let's conduct a more direct hypothesis test using simulation. Recall:

$$
H_0: \lambda = 3
$$

$$
H_1: \lambda \neq 3
$$

Pick a different test statistic. Obtain an empirical distribution of that test statistic under $H_0$. Next, find the $p$-value by determining how often this test statistic is at or further away from the test statistic derived from the sample. Remember that this is a two-sided test. 

##### Solution
I've added the bold.. because I could. We proceed. Since the problem states to pick a different test statistic, but then tells you which one to use, we will use the one we are supposed to use. That is, how often, with an emperical distribution, do we get values further away than the test statistic. Let's be kind to ourselves: the mean.

Let's break this down into nice breaks and steps. First, let's simulate under the null hypothesis (ie create a data set knowing this)

In [41]:
Simulated=np.array([np.mean(stats.expon.rvs(size=20,scale=1/3)) for _ in np.arange(10000)])
print(Simulated[1],Simulated[2],Simulated[3])

0.3388176589362678 0.3160561506072931 0.26371500316897933


Now that we have a set of simulated data, specifically the mean. Now, let's find the mean of the empirical data we have (the mean of my_data)

In [42]:
mu_data=np.mean(my_data)
print(mu_data)

0.1965


Here's where a nuance steps in, the average for the simulated should by 1/3, as determined by the maximum likelihood of a set with an exponential distribution. So, how many times is our simulated data set further away from 1/3 than 1/3 is from the $\mu_{my_{data}}$. So we are looking for, how many times is the simulated data set less than $\mu_{my_{data}}$ or .1965 and greater than 1/3+.1965. We proceed.

In [45]:
p=np.mean((Simulated)<0.1965)+np.mean(Simulated>(.1965+1/3))
print(p)

0.031


In [51]:
np.mean(Simulated>(.1965+1/3))

0.0123

How did the $p$-value compare to the LRT $p$-value? I wonder how the power of this test compares to our LRT. 

This p value is .031, which is slightly less of a p value than our LRT. We proceed to determining the power of this test...

#### Power

Let's figure out the power of this test. First, determine for what values of the test statistic would lead us to reject $H_0$. These values can be referred to as your rejection region. 

In [50]:
Upper=percentile(.95,Simulated)
print(Upper)
lower=percentile(.05,Simulated)
print(lower)

0.18516419499586506
0.12352416028542082


Now, determine the power of this test. Like in the LRT case, obtain a sample of size 20 and obtain the test statistic. Repeat many times and see how often your test statistic is in your rejection region. 

In [52]:
Simulated=np.array([np.mean(stats.expon.rvs(size=20,scale=1/3)) for _ in np.arange(10000)])
print(Simulated[1],Simulated[2],Simulated[3])

0.2424744210625122 0.3021744342403682 0.2794058156746516


Repeat for a sample size of 50. Note that you will have to obtain new critical values in order to do this.  

In [7]:
...

Ellipsis